<br><img src='https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRbAv_1oruFnK5S-AUSqKcRrpziHk8VQ1I1ujnTXccysijk5j7XBxPfdGXSUl04bkI3KNY&usqp=CAU' width='250' />

# Multiple Dashboard 

---
### Import the libraries and dataset

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from dash import dash, html, dcc
from dash.dependencies import Input, Output

In [ ]:
# import the dataset
def importdata() :
    var = pd.read_csv('airline_data.csv')
    return var

if __name__ == '__main__' :
    airline_data = importdata()

---
### Create the html skeleton

In [ ]:
# Create a dash application
app = dash.Dash(__name__)

app.layout = html.Div(children=[
    html.H1('Flight Delay Time Statistics',style={'text-align':'center','color':'#503D36','fontSize':40}),
    # Dash component
    html.Div(["Input Year:",dcc.Input(id='input-year',type='number',value='2010',style={'height':'35px','fontsize':30})
             ],style={'fontsize':30}),
    html.Br(), # Break point
    html.Br(),
    # Segment 1
    html.Div([
        html.Div(dcc.Graph(id='carrier-plot')),
        html.Div(dcc.Graph(id='weather-plot')),
    ],style={'display':'flex'}),
    html.Br(),
    html.Br(),
    # Segment 2
    html.Div([
        html.Div(dcc.Graph(id='nas-plot',figure={})),
        html.Div(dcc.Graph(id='security-plot')),
    ],style={'display':'flex'}),
    html.Br(),
    # Segment 3
    html.Div(dcc.Graph(id='late-plot'),style={'width':'65%'})
    ])

def compute_info(airline_data, entered_year):
    # Select data
    df =  airline_data[airline_data['Year']==int(entered_year)]
    # Compute delay averages
    avg_car = df.groupby(['Month','Reporting_Airline'])['CarrierDelay'].mean().reset_index()
    avg_weather = df.groupby(['Month','Reporting_Airline'])['WeatherDelay'].mean().reset_index()
    avg_NAS = df.groupby(['Month','Reporting_Airline'])['NASDelay'].mean().reset_index()
    avg_sec = df.groupby(['Month','Reporting_Airline'])['SecurityDelay'].mean().reset_index()
    avg_late = df.groupby(['Month','Reporting_Airline'])['LateAircraftDelay'].mean().reset_index()
    return avg_car, avg_weather, avg_NAS, avg_sec, avg_late

# Callback function
@app.callback([Output(component_id='carrier-plot', component_property='figure'),
               Output(component_id='weather-plot', component_property='figure'),
               Output(component_id='nas-plot', component_property='figure'),
               Output(component_id='security-plot', component_property='figure'),
               Output(component_id='late-plot', component_property='figure')
              ],
             Input(component_id='input-year', component_property='value'))

# Plot the graphs
def get_graph(input_year) :
    # Get the datas for the specific year
    avg_car, avg_weather, avg_NAS, avg_sec, avg_late = compute_info(airline_data,input_year)
    # plot the graph for carrier delay
    carrier_fig = px.scatter(avg_car,x='Month',y='CarrierDelay',
                          title='Average Carrier Delay Time (minutes) by Airline', 
                          color='Reporting_Airline')
    # plot the graph for Weather delay
    weather_fig = px.scatter(avg_weather,x='Month',y='WeatherDelay',
                          title='Average Weather Delay Time (minutes) by Airline', 
                          color='Reporting_Airline')
    # plot the graph for NAS delay
    nas_fig = px.scatter(avg_NAS,x='Month',y='NASDelay',
                          title='Average NAS Delay Time (minutes) by Airline', 
                          color='Reporting_Airline')
    # plot the graph for the security delay
    security_fig = px.scatter(avg_sec,x='Month',y='SecurityDelay',
                          title='Average Security Delay Time (minutes) by Airline', 
                          color='Reporting_Airline')
    # plot the graph for late aircraft delay
    late_fig = px.scatter(avg_late,x='Month',y='LateAircraftDelay',
                          title='Average Late Aircraft Delay Time (minutes) by Airline', 
                          color='Reporting_Airline')
    
    return [carrier_fig,weather_fig,nas_fig,security_fig,late_fig]

# Run the app
if __name__=='__main__':
    app.run_server()

---
<h1><center> END